In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

batch_size = 100
n_batch = mnist.train.num_examples // batch_size
lr = 1e-4
#initial weight function
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

#initial biases function
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#Convolutional Layer
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#Pooling Layer
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#Input data
with tf.name_scope('Input_data'):
    x = tf.placeholder(tf.float32,[None,784],name='x_input')
    y = tf.placeholder(tf.float32,[None,10],name='y-input')

#image reshape
x_image = tf.reshape(x,[-1,28,28,1])

#convolution layer1
with tf.name_scope('convolution_layer1'):
    with tf.name_scope('weigth_of_conv1'):
        W_conv1 = weight_variable([5,5,1,32])
    with tf.name_scope('baises_of_conv1'):
        b_conv1 = bias_variable([32])
    with tf.name_scope('conv1'):
        h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)
    with tf.name_scope('pool1'):
        h_pool1 = max_pool_2x2(h_conv1)

#convolution layer2
with tf.name_scope('convolution_layer2'):
    with tf.name_scope('weigth_of_conv2'):
        W_conv2 = weight_variable([5,5,32,64])
    with tf.name_scope('baises_of_conv2'):
        b_conv2 = bias_variable([64])
    with tf.name_scope('conv2'):
        h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
    with tf.name_scope('pool2'):
        h_pool2 = max_pool_2x2(h_conv2)
        
#Full connection layer
with tf.name_scope('Full_connection_layer1'):
    with tf.name_scope('W_fc1'):
        W_fc1 = weight_variable([7*7*64,1024])
    with tf.name_scope('b_fc1'):
        b_fc1 = bias_variable([1024])
    
    h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
    with tf.name_scope('h_fc1'):
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)
    
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)
    
#Full connection layer2
with tf.name_scope('Prediction'):
    with tf.name_scope('W_fc2'):
        W_fc2 = weight_variable([1024,10])
    with tf.name_scope('b_fc2'):
        b_fc2 = bias_variable([10])
    with tf.name_scope('h_fc2'):
        prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)
    
#Cross Entropy
with tf.name_scope('Cross_Entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',cross_entropy)
    
#optimizer
with tf.name_scope('optimizer'):
    train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)
    
#Accuracy
with tf.name_scope('Accuracy'):
    correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
#Session Run time
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # Writer graph to tensorborad
    tf.summary.FileWriter('./logs/CNN',tf.get_default_graph())
    
    merge = tf.summary.merge_all()
    summary_scale = tf.summary.FileWriter('./logs/CNN')
    # train step
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
            
        summary,acc = sess.run([merge,accuracy],feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        summary_scale
        print ('epoch:%d,Testing Accuracy:%f' %(epoch,acc))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
epoch:0,Testing Accuracy:0.947600
epoch:1,Testing Accuracy:0.971100
epoch:2,Testing Accuracy:0.977200
epoch:3,Testing Accuracy:0.982800
epoch:4,Testing Accuracy:0.983800
epoch:5,Testing Accuracy:0.985700
epoch:6,Testing Accuracy:0.986000
epoch:7,Testing Accuracy:0.986600
epoch:8,Testing Accuracy:0.988400
epoch:9,Testing Accuracy:0.988400
epoch:10,Testing Accuracy:0.989200
epoch:11,Testing Accuracy:0.989100
epoch:12,Testing Accuracy:0.989800
epoch:13,Testing Accuracy:0.989600
epoch:14,Testing Accuracy:0.991100
epoch:15,Testing Accuracy:0.989600
epoch:16,Testing Accuracy:0.990600
epoch:17,Testing Accuracy:0.990400
epoch:18,Testing Accuracy:0.991300
epoch:19,Testing Accuracy:0.991300
epoch:20,Testing Accuracy:0.991000
